## Índice o que?

O índice sharpe é um indicador usado para medir o retorno/risco de um determinado investimento comparado ao outro. Ele compara os retornos esperados para duas oportunidades de investimento e calcula o retorno adicional por unidade de risco que um investidor poderia obter escolhendo um sobre o outro. Em particular, analisa a diferença nos retornos de dois investimentos e compara a diferença média com o desvio padrão (como uma medida de risco) dessa diferença. Um índice Sharpe mais alto significa que a recompensa será maior por uma determinada quantidade de risco. É comum comparar uma oportunidade específica com uma referência que representa uma categoria inteira de investimento.
O índice sharpe usualmente é usado para comparar um portifolio de ações e um investimento "livre de riscos" como benchmark. Para simplicar a analise vou utilizar duas ações populares da B3 em 2019, são a da Magazine Luiza(MGLU3) e a da Via Varejo(VVAR3) e ao invés de usar um investimento "sem risco", vou usar o índice Ibovespa pela facilidade de encontrar os valores diários de ambos no Google finance, não sendo necessário um conversão de um indice anual como alguma modalidade do tesouro direto. Usando o indice em vez do investimento sem risco obteremos das ações qual obteve o maior retorno em uma unidade de risco comparado ao IBOV.


P.S Essa análise foi inspirada no projeto "Risk and Returns: The Sharpe Ratio" do DataCamp

COmo calcular: https://clubedovalor.com.br/indice-de-sharpe/

In [27]:
# Importando os módulos necessários
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

#Lendo o dataset criado através do google finance
stock_data = pd.read_csv("datasets/stocks.csv",parse_dates = True, index_col = 'DATE').dropna()
benchmark_data = pd.read_csv("datasets/benchmark.csv",parse_dates = True, index_col = 'DATE').dropna()

## 2. Dando uma olhada nos dados

Antes de começar da uma olhada se os dados coletados pelo google finance estão corretos e todos os 3 seguem o mesmo padrão

In [28]:
# Visualizando o sumário para stock_data
print('Stocks\n')
print(stock_data.info())
print(stock_data.head())
# Display summary for benchmark_data
print('\nBenchmarks\n')
print(benchmark_data.info())
print(benchmark_data.head())

Stocks

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 248 entries, 2019-01-02 to 2019-12-30
Data columns (total 2 columns):
VVAR     248 non-null float64
MGLU3    248 non-null float64
dtypes: float64(2)
memory usage: 5.8 KB
None
            VVAR  MGLU3
DATE                   
2019-01-02  4.38  23.25
2019-01-03  4.31  23.00
2019-01-04  4.27  22.30
2019-01-07  4.24  22.65
2019-01-08  4.04  22.46

Benchmarks

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 248 entries, 2019-01-02 to 2019-12-30
Data columns (total 1 columns):
IBOV    248 non-null float64
dtypes: float64(1)
memory usage: 3.9 KB
None
                IBOV
DATE                
2019-01-02  91012.31
2019-01-03  91564.25
2019-01-04  91840.79
2019-01-07  91699.05
2019-01-08  92031.86


## 3 . Plotando os preços diarios da Magalu & Via Varejo

Antes de comparar o investimento na Magazine Luiza ou Via Varejo com o Ibovespa, é interessante visualizar os dados para entende-los melhor, para todos os plots usarei o Plotly


In [29]:
cf.go_offline()

stock_data.iplot(kind = "scatter", filename = "lineplot-with-cufflinks", title='MGLU3 Vs VVAR3 2019')


In [30]:
cf.go_offline()

benchmark_data.iplot(kind = "scatter", filename = "lineplot-with-cufflinks", title='IBovespa 2019')

## 4 Os inputs para o índice sharpe: Iniciando com o retorno diário das ações

O índice Sharpe usa a diferença de retorno entre duas oportunidades de investimento em consideração. No entanto, os dados de ações mostram o valor histórico de cada investimento, não o retorno. Para calcular o retorno, preciso calcular a alteração percentual no calor de um dia para o outro. Vou olhar também os dados estatísticos resumidos porque alguns deles serão usados para calcular o indíce sharpe.


In [31]:
# Calculando o retorno diário do stock_data
stock_returns = stock_data.pct_change()

# Plotando o retorno diário
stock_returns.iplot(title='Retorno diário em 2019: MGLU3 Vs VVAR3 ')


# Olhando os dados estatisticos do retorno diário da ações
stock_returns.describe()

,VVAR,MGLU3
count,247.000000,247.000000
mean,0.004327,0.003243
std,0.032722,0.025859
min,-0.103647,-0.057745
25%,-0.015962,-0.012970
50%,0.002488,0.001774
75%,0.022829,0.017282
max,0.102244,0.104270


In [32]:
# Calculando o retorno diário do benchmark_data
ibov_returns = benchmark_data['IBOV'].pct_change()

# Plotando o retorno diário
ibov_returns.iplot(kind = "scatter", filename = "lineplot-with-cufflinks", title='Retorno diário 2019 Ibovespa')


# Olhando os dados estatisticos do retorno diário do Ibovespa
ibov_returns.describe()

count    247.000000
mean       0.001032
std        0.011139
min       -0.037388
25%       -0.005138
50%        0.001803
75%        0.008469
max        0.027911
Name: IBOV, dtype: float64

## 5. Calculando o excesso do retorno para a Magalu e Via varejo vs Ibovespa

Agora é necessário calcular a performance relativa da ações vs o Ibovespa. Isso é feito calculando a diferença de retorno entre <code>stock_returns</code> e <code>sp_returns</code> para cada dia.

In [33]:
# Calculando a diferença no retorno diário
excess_returns = stock_returns.sub(ibov_returns, axis =0)

# plotando o excesso da diferença
excess_returns.iplot(kind = "scatter", filename = "lineplot-with-cufflinks", title='Excesso de Retorno Magalu e Via Varejo Vs Ibovespa')


# Olhando os dados estatisticos do excesso do retorno diário
excess_returns.describe()


,VVAR,MGLU3
count,247.000000,247.000000
mean,0.003294,0.002211
std,0.030327,0.022398
min,-0.107660,-0.052439
25%,-0.016129,-0.013928
50%,0.000780,-0.000233
75%,0.019178,0.016939
max,0.100184,0.094435


## 6. Finalmente o  Índice Sharpe, primeiro passo: A diferença média de retorno diário das ações vs IBOV 

Agora podemos calcular o Índice Sharpe. Primeiro preciso calcular a média do <code>excess_returns</code>. Isso me diz se o investimento rendeu mais ou menos comparado com o Ibovespa.

In [34]:
# Calculando a média do  excess_returns 

avg_excess_return = excess_returns.mean()

# plotando o avg_excess_returns
avg_excess_return.iplot(kind = "bar", filename = "lineplot-with-cufflinks", title='Excesso do retorno IBOV')


## 6. O  Índice Sharpe, segundo passo:  O desvio padrão da diferença do retorno

Percebe-se uma diferença considerável entre a média de retorno diário para Magalu e Via Varejo

Agora, preciso calcula o desvio padrão do <code>excess_returns</code>. Isso mostra a quantidade de risco que o investimento em ações tem comparado com o investimento na Ibovespa.

In [36]:
# calculando o desvio padrão
sd_excess_return = excess_returns.std()

# plotando os desvios padrões
sd_excess_return.iplot(kind= "bar" , title='Desvio padrão da diferença do retorno')

## 7 Colocando tudo junto

Agora eu só preciso calcular a proporção <code>avg_excess_returns</code> e <code>sd_excess_returns</code>. o resultado é finalmente o indíce Sharpe e indica se ocorreu mais (ou menos) retorno sobre o investimento considerando o redimento por unidade de risco.
O Índice Sharpe é frequentemente anualizado multiplicando-o pela raiz quadrada do números de períodos. Utilizamos dados diários, portanto usaremos a raiz de 248 que foram a quantidade de dias que ocorreram pregões na Ibovespa em 2019

In [37]:
# Calculando o Índice Sharpe siário
daily_sharpe_ratio = avg_excess_return.div(sd_excess_return)

# Anualizando o Índice
annual_factor = np.sqrt(248) 
annual_sharpe_ratio = daily_sharpe_ratio.mul(annual_factor)

# plotando o índice Sharpe anualizado
annual_sharpe_ratio.iplot(kind ="bar", title=' Sharpe Ratio Anual: Ações vs IBOV')

## 8. Conclusões

Dados os 2 índices Sharpe, qual investimento devemos optar? Em 2019 a Via Varejo tem um índice Sharpe maior que a Magalu. Isso significa que um investimento na Via Varejo teve um retorno melhor comparado com o Ibovespa para cada unidade de risco que o investidor assumiu. Em outras palavras o investimento na VVAR3 foi mais atrativo.

A diferença foi motivada principalmente pelas diferenças de retorno da VVAR e MGLU.  Quanto confrontado com alternativas de investimento que oferencem retornos e riscos diferente, o índice Sharpe ajuda a tomar uma decisão  ajustando os retornos pelas diferenças de risco e permite que o investidor compare oportunidades de investimento em termos de igualdade.

